In [17]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("DataFrameSQL").getOrCreate()

Load Data into a DataFrame

In [18]:
# Load the synthetic data into a DataFrame
data_file_path = "C:/Users/rammo/OneDrive/Desktop/person.csv"
df = spark.read.csv(data_file_path, header=True, inferSchema=True)

In [19]:
# Display schema of DataFrame
df.printSchema()

# Show the initial DataFrame
print("Initial DataFrame:")
df.show(1)

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)

Initial DataFrame:
+----+---+------+------+
|name|age|gender|salary|
+----+---+------+------+
|John| 30|     M|50,000|
+----+---+------+------+
only showing top 1 row



Register the DataFrame as a Temporary Table

In [20]:
# Register the DataFrame as a Temporary Table
df.createOrReplaceTempView("my_table")

Perform SQL-like Queries

In [21]:
# Select all rows where age is greater than 25
result = spark.sql("SELECT * FROM my_table WHERE age > 25")

result.show()

+-------+---+------+------+
|   name|age|gender|salary|
+-------+---+------+------+
|   John| 30|     M|50,000|
|  David| 35|     M| 52000|
|  Emily| 28|     F| 75000|
|Michael| 40|     M| 50000|
|  sarah| 32|     F| 51000|
+-------+---+------+------+



In [22]:
# Compute the average salary by gender
avg_salary_by_gender = spark.sql("SELECT gender, AVG(salary) as avg_salary FROM my_table GROUP BY gender")
avg_salary_by_gender.show()

+------+----------+
|gender|avg_salary|
+------+----------+
|     F|   63000.0|
|     M|   51000.0|
+------+----------+



Creating and managing temporary views.

In [23]:
# Create a temporary view
df.createOrReplaceTempView("people")

In [24]:
# Query the temporary view
result = spark.sql("SELECT * FROM people WHERE age > 25")

result.show()

+-------+---+------+------+
|   name|age|gender|salary|
+-------+---+------+------+
|   John| 30|     M|50,000|
|  David| 35|     M| 52000|
|  Emily| 28|     F| 75000|
|Michael| 40|     M| 50000|
|  sarah| 32|     F| 51000|
+-------+---+------+------+



In [25]:
# Check if a temporary view exists
view_exists = spark.catalog.tableExists("people")
view_exists

True

In [26]:
# Drop a temporary view
spark.catalog.dropTempView("people")

True

In [28]:
# Check if a temporary view exists
view_exists = spark.catalog.tableExists("people")

In [29]:
view_exists

False

Subquries

In [30]:
# Create DataFrames
employee_data = [
    (1, "John"), (2, "Alice"), (3, "Bob"), (4, "Emily"),
    (5, "David"), (6, "Sarah"), (7, "Michael"), (8, "Lisa"),
    (9, "William")
]
employees = spark.createDataFrame(employee_data, ["id", "name"])

salary_data = [
    ("HR", 1, 60000), ("HR", 2, 55000), ("HR", 3, 58000),
    ("IT", 4, 70000), ("IT", 5, 72000), ("IT", 6, 68000),
    ("Sales", 7, 75000), ("Sales", 8, 78000), ("Sales", 9, 77000)
]
salaries = spark.createDataFrame(salary_data, ["department", "id", "salary"])

employees.show()

salaries.show()

+---+-------+
| id|   name|
+---+-------+
|  1|   John|
|  2|  Alice|
|  3|    Bob|
|  4|  Emily|
|  5|  David|
|  6|  Sarah|
|  7|Michael|
|  8|   Lisa|
|  9|William|
+---+-------+

+----------+---+------+
|department| id|salary|
+----------+---+------+
|        HR|  1| 60000|
|        HR|  2| 55000|
|        HR|  3| 58000|
|        IT|  4| 70000|
|        IT|  5| 72000|
|        IT|  6| 68000|
|     Sales|  7| 75000|
|     Sales|  8| 78000|
|     Sales|  9| 77000|
+----------+---+------+



In [31]:
# Register as temporary views
employees.createOrReplaceTempView("employees")
salaries.createOrReplaceTempView("salaries")

In [32]:
# Subquery to find employees with salaries above average
result = spark.sql("""
    SELECT name
    FROM employees
    WHERE id IN (
        SELECT id
        FROM salaries
        WHERE salary > (SELECT AVG(salary) FROM salaries)
    )
""")

result.show()

+-------+
|   name|
+-------+
|  Emily|
|  David|
|Michael|
|   Lisa|
|William|
+-------+



Window Function

In [35]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

 Window function from pyspark.sql.window is used to perform advanced data operations such as ranking, cumulative aggregations, and moving calculations over a specified range of rows in a DataFrame. This functionality is crucial for complex data transformations and analytics tasks.

1.row_number(): Assigns a unique number to each row within a partition.
2.rank(): Assigns a rank to each row within a partition, with gaps for ties.
3.dense_rank(): Assigns ranks with no gaps for ties.
4.lead(): Accesses the next row's value in the partition.
5.lag(): Accesses the previous row's value in the partition.
6.sum(): Computes the sum of values within the window.
7.avg(): Computes the average of values within the window.
8.max() / min(): Computes the maximum or minimum value within the window.

In [36]:
employee_salary = spark.sql("""
    select  salaries.*, employees.name
    from salaries 
    left join employees on salaries.id = employees.id
""")

employee_salary.show()

+----------+---+------+-------+
|department| id|salary|   name|
+----------+---+------+-------+
|        HR|  1| 60000|   John|
|        HR|  2| 55000|  Alice|
|        HR|  3| 58000|    Bob|
|        IT|  4| 70000|  Emily|
|        IT|  6| 68000|  Sarah|
|        IT|  5| 72000|  David|
|     Sales|  7| 75000|Michael|
|     Sales|  9| 77000|William|
|     Sales|  8| 78000|   Lisa|
+----------+---+------+-------+



In [37]:
# Create a window specification
window_spec = Window.partitionBy("department").orderBy(F.desc("salary"))

In [38]:
# Calculate the rank of employees within each department based on salary

employee_salary.withColumn("rank", F.rank().over(window_spec)).show()

+----------+---+------+-------+----+
|department| id|salary|   name|rank|
+----------+---+------+-------+----+
|        HR|  1| 60000|   John|   1|
|        HR|  3| 58000|    Bob|   2|
|        HR|  2| 55000|  Alice|   3|
|        IT|  5| 72000|  David|   1|
|        IT|  4| 70000|  Emily|   2|
|        IT|  6| 68000|  Sarah|   3|
|     Sales|  8| 78000|   Lisa|   1|
|     Sales|  9| 77000|William|   2|
|     Sales|  7| 75000|Michael|   3|
+----------+---+------+-------+----+



In [39]:
# Stop the SparkSession
spark.stop()